# Обнуление маловажной статистики
**На основе изменения множественной регрессии** (единственный Y)

# Импорт библиотек

In [ ]:
import numpy as np
import pylab as plb
from scipy.stats import linregress
import pandas as pd

import statsmodels.api as sm
import pandas.util.testing as tm

import copy

# Загрузка и обработка статистики

In [ ]:
# Источник

url = "https://raw.githubusercontent.com/ForestTiger-GH/COTRA/main/Stat_wheat/Stat_wheat_alldates516_trend20.27.1_nonfilter.txt"

downloaded_url = pd.read_csv(url,
               sep='\t', 
               error_bad_lines=False,
               decimal=',',
               encoding='gbk', 
               skiprows=1)

     10.08.2010  0,4453530  0,4559149  ...  0,2663208  0,7886993   6
0    17.08.2010   0.428423  -0.192284  ...   0.322736   0.741482  10
1    24.08.2010   0.413532  -0.109575  ...   0.349325   0.702611  10
2    31.08.2010   0.292744  -0.044152  ...   0.384214   0.637684  12
3    07.09.2010   0.234552   0.038211  ...   0.283104   0.499665   7
4    14.09.2010   0.188608  -0.004603  ...   0.299126   0.441848   7
..          ...        ...        ...  ...        ...        ...  ..
510  02.06.2020  -0.698913  -0.590263  ...  -0.568573  -0.701205   9
511  09.06.2020  -0.668454  -0.733715  ...  -0.546280  -0.764128  10
512  16.06.2020  -0.538811  -0.629503  ...  -0.552218  -0.735286  13
513  23.06.2020  -0.360195  -0.656658  ...  -0.490171  -0.708247  15
514  30.06.2020  -0.228664  -0.612057  ...  -0.471837  -0.685974  16

[515 rows x 29 columns]


In [ ]:
#downloaded_url.to_numpy()
a = np.asarray(downloaded_url)

columns_all = np.array(a).T

column_dates = columns_all[0].tolist()
width_all = len(a[0])                                                            # Количество всех столбцов
width_y = 1                                                                      # Количество Y (всегда 1)
width_x = width_all - 1 - width_y                                                # Количество Х

column_dates = columns_all[0]                                                    # Столбец дат
np.array(column_dates)
column_y = columns_all[width_all-1]                                              # Столбец Y
np.array(column_y)

lenght_all = len(column_dates)                                                   # Количество строк (на основе дат)

In [ ]:
columns_x_all = np.array(columns_all[1:(len(columns_all)-1)]).tolist()           # Группа столбцов X
np.array(columns_x_all).tolist()

lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
columns_x_all = lol(columns_x_all, width_x)
columns_x_all = columns_x_all[0]

columns_x_all_old = copy.deepcopy(columns_x_all)

# Уровень check rate

In [ ]:
check_rate = 0.1                                                                 # Точность проверки (0.1 или 0.01)
check_rate_amount = round(1/check_rate)                                             # то есть 10 вариаций или 100 
check_rate_list = (np.arange(0, 1, check_rate)).tolist()                         # Список уровней (проверка будет на каждом)
check_rate_count = (np.arange(0, check_rate_amount, 1)).tolist()

# Обработка данных

In [ ]:
def reg_m(yyy, xxx):
    ones = np.ones(len(xxx[0]))
    X = sm.add_constant(np.column_stack((xxx[0], ones)))
    for ele in xxx[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    results = sm.OLS(yyy, X).fit()
    return results

In [ ]:
# Обработка Х-ов и расчет коэффициентов корреляции

group_x = []
best_positions = []
list_Rsquare_saved = []

column_x_old = []

for q in range(width_x):
    column_x = columns_x_all[q]
    column_x_old = column_x
    column_x_new = column_x

    x_step_value = max(np.abs(column_x).tolist())/check_rate_amount              # Шаг отсечения, исходя из реальной статы Х-ов

    border_x = []
    for e in range(check_rate_amount):
        border_x.append(check_rate_list[e]*x_step_value*check_rate_amount)       # Сформированы бордеры для Х-ов
    
    list_Rsquare = []
    best_Rsquare = []
    for r in range(check_rate_amount):
        for t in range(lenght_all):
            if abs(column_x[t]) < border_x[r]:
                column_x[t] = 0                                                  # Сформирован новый столбец Х
        
        columns_x_all[q] = column_x
        yyy = column_y
        xxx = columns_x_all
        yyy = np.array(yyy,'float32')
        xxx = np.array(xxx,'float32')
        list_Rsquare.append(reg_m(yyy, xxx).rsquared)
    list_Rsquare_saved.append(np.array(list_Rsquare))                            # Сохранение R-квадратов

    if abs(max(list_Rsquare)) >= abs(min(list_Rsquare)):
        best_Rsquare = max(list_Rsquare)
    else:
        best_Rsquare = min(list_Rsquare)

    best_position_number = list_Rsquare.index(best_Rsquare)
    best_positions.append(best_position_number)

    for n in range(len(column_x_new)):
        if column_x_new[n] < border_x[best_position_number]:
            column_x_new[n] = 0

    group_x.append(column_x_new)

# Промежуточные результаты

In [ ]:
# Лучшие уровни отсечки (в процентах*Check_rate)

print(best_positions)

[8, 9, 8, 7, 7, 2, 4, 8, 2, 6, 7, 0, 1, 3, 7, 6, 8, 2, 6, 9, 0, 5, 7, 9, 9, 5, 6]


In [ ]:
# Матрица результатов регрессии: R-квадрат

print(list_Rsquare_saved)

[array([0.45416017, 0.45303543, 0.45190629, 0.45188599, 0.45413836,
       0.45226219, 0.4534265 , 0.45848866, 0.46404441, 0.45182092]), array([0.45186314, 0.4538108 , 0.4529028 , 0.45187524, 0.45322817,
       0.45802247, 0.46471392, 0.46464841, 0.46874301, 0.47756851]), array([0.46887092, 0.46898844, 0.46811897, 0.47138869, 0.46906857,
       0.4693375 , 0.47011829, 0.47542855, 0.47692095, 0.46854449]), array([0.46800459, 0.46821231, 0.4682307 , 0.47143336, 0.46870271,
       0.46840487, 0.46899634, 0.47661617, 0.4718305 , 0.46774939]), array([0.46750253, 0.46746271, 0.46827165, 0.46855613, 0.46755725,
       0.47043641, 0.46843502, 0.47438039, 0.46874054, 0.46919083]), array([0.46750897, 0.46765598, 0.46805328, 0.46723371, 0.46495649,
       0.46491794, 0.46248527, 0.46012686, 0.45950302, 0.46046605]), array([0.46127056, 0.46002433, 0.46215412, 0.46433095, 0.47449975,
       0.4738177 , 0.46341617, 0.45585247, 0.44016567, 0.438383  ]), array([0.44093535, 0.44010857, 0.43904334, 0.43

In [ ]:
print(reg_m(np.array(column_y,'float32'), np.array(columns_x_all_old,'float32')).summary())
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print(reg_m(np.array(column_y,'float32'), np.array(group_x,'float32')).summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.425
Method:                 Least Squares   F-statistic:                     15.62
Date:                Wed, 10 Mar 2021   Prob (F-statistic):           7.27e-49
Time:                        21:43:01   Log-Likelihood:                -1695.5
No. Observations:                 515   AIC:                             3445.
Df Residuals:                     488   BIC:                             3560.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.1480      1.766      1.783      0.0

# Вывод статистики

In [ ]:
columns = []
#columns.append("Dates")        ### пока без дат
for b in range(width_x):
    alfa = "Variable {}".format(str(b+1))
    columns.append(alfa)
    alfa = []
columns.append("Dependent Variable 1")

['Variable 1',
 'Variable 2',
 'Variable 3',
 'Variable 4',
 'Variable 5',
 'Variable 6',
 'Variable 7',
 'Variable 8',
 'Variable 9',
 'Variable 10',
 'Variable 11',
 'Variable 12',
 'Variable 13',
 'Variable 14',
 'Variable 15',
 'Variable 16',
 'Variable 17',
 'Variable 18',
 'Variable 19',
 'Variable 20',
 'Variable 21',
 'Variable 22',
 'Variable 23',
 'Variable 24',
 'Variable 25',
 'Variable 26',
 'Variable 27',
 'Dependent Variable 1']

In [ ]:
beta = np.array(group_x).T
merged_one = []
merged_all = []

np.array(column_dates)

for g in range(len(beta)):
    merged_one = []
    merged_one.append(beta[g].tolist())
    merged_one.append(column_y[g])
    np.array(merged_one)
    merged_all.append(merged_one)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
csv_half_ready = []
csv_ready = []

for k in range(len(merged_all)):
      for f in range(len(merged_all[0][0])):
          csv_half_ready.append(merged_all[k][0][f])
      csv_half_ready.append(merged_all[k][1])
      np.array(csv_half_ready)
      csv_ready.append(csv_half_ready)
      csv_half_ready = []


In [ ]:
data = csv_ready
df = pd.DataFrame(data, columns=columns)
df.to_csv(index=False)

'Variable 1,Variable 2,Variable 3,Variable 4,Variable 5,Variable 6,Variable 7,Variable 8,Variable 9,Variable 10,Variable 11,Variable 12,Variable 13,Variable 14,Variable 15,Variable 16,Variable 17,Variable 18,Variable 19,Variable 20,Variable 21,Variable 22,Variable 23,Variable 24,Variable 25,Variable 26,Variable 27,Dependent Variable 1\n0.0,0.0,0.0,0.0,0.8458216000000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8861064000000001,0.0,0.0,0.0,0.0,0.7414818,10\n0.0,0.0,0.0,0.0,0.8400421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8696814,0.0,0.0,0.0,0.0,0.0,10\n0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8586231,0.0,0.0,0.0,0.0,0.0,12\n0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8443761999999999,0.0,0.0,0.0,0.0,0.0,7\n0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8265061,0.0,0.0,0.0,0.0,0.0,7\n0.0,0.0,0.0,0.0,0.0,0.0,0.0,0